# Basic setup

Here we will import the `pyspark` module and set up a `SparkSession`.  By default, we'll use a `SparkSession` running locally, with one Spark executor; we're dealing with small data, so it doesn't make sense to run against a cluster.


In [1]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession, SQLContext

spark = SparkSession.builder.master("local[1]").getOrCreate()
sc = spark.sparkContext

In [2]:
nodes = spark.read.load("/data/nodes.parquet").withColumnRenamed("_1", "id").withColumnRenamed("_2", "Wallet")
nodes.show(5)

+---+--------------------+
| id|              Wallet|
+---+--------------------+
|  0|bitcoinaddress_93...|
|  1|bitcoinaddress_4D...|
|  2|bitcoinaddress_BE...|
|  3|bitcoinaddress_4B...|
|  4|bitcoinaddress_44...|
+---+--------------------+
only showing top 5 rows



In [3]:
edges = spark.read.load("/data/edges.parquet").withColumnRenamed("srcId", "src").withColumnRenamed("dstId", "dst")

In [4]:
edges.show(5)

+------+------+-----+
|   src|   dst| attr|
+------+------+-----+
|150102|107378|input|
|470403|107378|input|
|232249| 97703|input|
|539070| 97703|input|
|131174|176711|input|
+------+------+-----+
only showing top 5 rows



## Constructing the graph representation

Spark contains API for graph processing. It's called [graphx](https://spark.apache.org/graphx/) and it also comes with multiple built-in algorithms like page-rank. It uses the [Pregel API](https://spark.apache.org/docs/latest/graphx-programming-guide.html#pregel-api).

In [5]:
from graphframes import *

g = GraphFrame(nodes, edges)

In [ ]:
results = g.pageRank(resetProbability=0.15, tol=0.01)
results.vertices.select("id", "pagerank").show()
results.edges.select("src", "dst", "weight").show()

# this will fail with OOM error